# Measurement Plan
## Calibration
To calibrate, once I have assembled my mosaic, I am going to compare it to a finder chart of NGC 2244 and attempt to find a star with know brightness to calibrate the image with. Since my current individual images are a bit small, it is difficult to identify what I'm looking at on a finder chart, so it should be easier when my mosaic is complete.
I will demonstrate how I will get the flux of a calibration star by getting the flux of a random star.

In [1]:
import numpy as np
import pylab as pl
sigma_to_fwhm = np.sqrt(8*np.log(2))
def photometry_and_astrometry(image, star_center, cutout_width,
                              estimated_dark_current=4, background=None,
                              pixel_uncertainty_estimate=None,
                              threshold=2,
                              show_cutout=False,  
                             ):    
    # blyc = bottom-left y-coordinate    
    # tryc = top-right y-coordinate    
    blyc = star_center[0]-cutout_width[0]//2
    tryc = star_center[0]+cutout_width[0]//2
    blxc = star_center[1]-cutout_width[1]//2
    trxc = star_center[1]+cutout_width[1]//2
    star_cutout = image[blyc:tryc, blxc:trxc]
    # Key modification    
    # image < 2*sigma    
    star_cutout_mask = star_cutout < threshold*pixel_uncertainty_estimate
    if show_cutout:
        pl.figure()
        pl.imshow(star_cutout)
        pl.colorbar()    
        
    yc, xc = np.indices(star_cutout.shape)
    mom1_x = (xc * star_cutout).sum() / star_cutout.sum()
    mom1_y = (yc * star_cutout).sum() / star_cutout.sum()
    mom2_x = ((xc - mom1_x)**2 * star_cutout).sum() / star_cutout.sum()
    mom2_y = ((yc - mom1_y)**2 * star_cutout).sum() / star_cutout.sum()
    fwhm_x = mom2_x**0.5*sigma_to_fwhm
    fwhm_y = mom2_y**0.5*sigma_to_fwhm
    star_cutout_including_background = star_cutout + background + estimated_dark_current
    readnoise_squared = 100
    sigma_squared_F = star_cutout_including_background + readnoise_squared
    F = star_cutout
    sigma_squared_xi = ((sigma_squared_F * xc**2).sum() / F.sum()**2 +
                        (xc*F).sum()**2 * sigma_squared_F.sum() / F.sum()**4)
    sigma_squared_yi = ((sigma_squared_F * yc**2).sum() / F.sum()**2 +
                        (yc*F).sum()**2 * sigma_squared_F.sum() / F.sum()**4)
    mom1_x_uncertainty = sigma_squared_xi**0.5
    mom1_y_uncertainty = sigma_squared_yi**0.5
    total_flux = star_cutout.sum()
    total_flux_uncertainty = (star_cutout + pixel_uncertainty_estimate**2 + readnoise_squared).sum()**0.5
    # in the steps above, we wanted to _include_ the star.  Now, in the whole image, we want to _exclude_ it!    
    star_cutout_mask = ~star_cutout_mask    
    # note that we want the coordinates in the original frame!    
    return (total_flux, total_flux_uncertainty, mom1_x+blxc, mom1_y+blyc,
            mom1_x_uncertainty, mom1_y_uncertainty, fwhm_x, fwhm_y)

In [2]:
import glob
from astropy.io import fits
image = glob.glob("example.fits")
example = [fits.getdata(fn) for fn in image][0]

exampleflux = photometry_and_astrometry(example, (249,73), (30,30),
                              estimated_dark_current=0, background=100,
                              pixel_uncertainty_estimate=0,
                              threshold=2,
                              show_cutout=False,  
                              )
exampleflux

(2605961.4810937555,
 1641.938330478266,
 73.16013586962137,
 249.82376491366918,
 0.013936093232986257,
 0.01457853511144033,
 7.768957952308102,
 9.442575715459823)

## Stellar Flux per Pixel
To make the pixel by pixel, color-color plot, I'm going to take my assembled mosaics for each filter, take their data, and create a plot using the code in the notebook.

I don't really know what to write for this one. I'm nowhere near being ready to work on it tonight. 

## Compose an RGB 
First, I will create a mosaic with all 6 images in each filter. Since I alright aligned each filter within an image, this speeds up the process since I will realistically only have to go through the process of coordinating how to mosaic the images together once and will not have to align the final images (hopefully). Then I will use make_lupton_rgb to make a color image.

While I have do have 2 images mosaiced together in each filter, it is not necessary for me to include the code for that since it wasn't one of the things you asked for. I hope you accept the numerous screenshots I sent while asking you for help as proof that I have gotten some mosaicing done.